# Import

In [1]:
import pandas as pd
import numpy as np
from io import StringIO

import os

from tqdm import tqdm

# Load Dataset

In [3]:
data_dir = 'dataset'

In [4]:
network_file = 'Network.csv'
network_path = os.path.join(data_dir, network_file)

In [5]:
network = pd.read_csv(network_path, encoding = 'cp949')

In [6]:
network.head(3)

,Init node,Term node,Free Flow Time,Power,Capacity,B,Length
0,1,2,1,4,1,0.15,1
1,1,3,1,4,3,0.15,1
2,3,2,1,4,2,0.15,1


# Cost 생성

In [7]:
network.columns

Index(['Init node ', 'Term node ', 'Free Flow Time ', 'Power', 'Capacity ',
       'B', 'Length '],
      dtype='object')

In [10]:
# O, D, FFT, alpha, Capacity, beta, Length, Toll 만 남기고 나머지는 지운다.
# Volume, Cost라는 빈 컬럼을 생성한다.
# cost를 구한다.
net1 = pd.DataFrame({
    'O' : network['Init node '],
    'D' : network['Term node '],
    'FFT' : network['Free Flow Time '],
    'alpha' : network['Power'],
    'Capacity' : network['Capacity '],
    'beta' : network['B'],
    'Length' : network['Length '],
    'Link_num' : [i for i in range(1, len(network)+1)],
    #'Toll' : network['Toll '],
    'Volume' : 0,
    'Cost' : 0
})

In [11]:
net1.head(3)

,O,D,FFT,alpha,Capacity,beta,Length,Volume,Cost
0,1,2,1,4,1,0.15,1,0,0
1,1,3,1,4,3,0.15,1,0,0
2,3,2,1,4,2,0.15,1,0,0


In [13]:
net1['Cost'] = net1['FFT'] * ((1 + net1['alpha']/net1['Capacity']) ** net1['beta'])# + (net1['Length'] * net1['Toll'])

In [14]:
Cost1 = pd.DataFrame({
    'From' : net1['O'],
    'To' : net1['D'],
    'cost' : net1['Cost']
})

In [15]:
Cost1.head(3)

,From,To,cost
0,1,2,1.273050
1,1,3,1.135525
2,3,2,1.179148


# Node set to find optimize

In [16]:
node1 = pd.DataFrame({'node' : net1['O'],
                      'start' : 0,
                      'end' : 0}).drop_duplicates()

node1 = node1.sort_values(by = 'node') # node 컬럼에 대해 오름차순 정렬

In [17]:
node1.head(3)

,node,start,end
0,1,0,0
2,3,0,0
4,4,0,0


In [18]:
# net1 데이터프레임을 O, D에 대하여 오름차순 정렬
net1 = net1.sort_values(by = ['O', 'D'])
# Cost를 일단 정수 값으로 설정하기
net1['Cost'] = net1['Cost'].round(0)

In [19]:
net1.head()

,O,D,FFT,alpha,Capacity,beta,Length,Volume,Cost
0,1,2,1,4,1,0.15,1,0,1.0
1,1,3,1,4,3,0.15,1,0,1.0
2,3,2,1,4,2,0.15,1,0,1.0
3,3,4,1,4,4,0.15,1,0,1.0
4,4,2,1,4,5,0.15,1,0,1.0


## 노드 넘버 설정
* 각 출발지로부터 뻗어나가는 링크에 번호매긴거

In [20]:
# 76개의 node = (O, D) 페어링에 대해 번호를 매겨주기
net1['link_num'] = [i for i in range(1, len(net1)+1)]

net1.head()

,O,D,FFT,alpha,Capacity,beta,Length,Volume,Cost,link_num
0,1,2,1,4,1,0.15,1,0,1.0,1
1,1,3,1,4,3,0.15,1,0,1.0,2
2,3,2,1,4,2,0.15,1,0,1.0,3
3,3,4,1,4,4,0.15,1,0,1.0,4
4,4,2,1,4,5,0.15,1,0,1.0,5


In [21]:
# 각 기점별 링크넘버 start-end를 node1이라는 데이터프레임에 저장하기
nodes = net1['O'].unique()
start_list = []
end_list = []

for node in nodes:
    des_list = net1[net1['O'] == node]['link_num']
    des_list.sort_values() # 오름차순 정렬
    start = des_list.iloc[0]
    end = des_list.iloc[-1]
    start_list.append(start)
    end_list.append(end)
    
node1 = pd.DataFrame({
    'node' : nodes,
    'start' : start_list,
    'end' : end_list
})

In [22]:
node1

,node,start,end
0,1,1,2
1,3,3,4
2,4,5,5


# Dijkstra

In [57]:
def Dijkstra(origin, net1):
    
    Pk = pd.DataFrame({
        'To' : net1['O'].unique(),
        'From' : 0,
        'C1' : 999,
        'Fin' : 0
    })
    
    Pk.loc[Pk['To'] == origin, 'C1'] = 0
    
    Pk = Pk.sort_values(by = ['C1', 'To']).reset_index(drop = True)
    print(f'PK :\n {Pk}')
    
    while True:
        if sum(Pk['Fin']) == len(Pk):
            break

        a = Pk.loc[0, 'To'] #Pk 데이터프레임의 1행 1열 데이터
        ac = Pk.loc[0, 'C1'] # Pk 데이터프레임의 1행 3열 데이터
        
        print(a)
        
        #print(a, ac)
        
        Pk.loc[0, 'Fin'] = 1
        Pk = Pk.sort_values(by = ['To']).reset_index(drop = True)
        
        print(f'PK :\n {Pk}')
            
        # DN : Dijkstra next updating node, D : Dijkstra iteration term
        node1_a = node1[node1['node'] == a]
        link_start = node1_a['start'].iloc[0]
        link_end = node1_a['end'].iloc[0]
        
        print(link_start, link_end)
         
        DN = net1[(net1['link_num'] >= link_start) & (net1['link_num'] <= link_end)]
        print(DN)
        DN = DN[['D', 'Cost']]
        DN.rename(columns = {'D' : 'To', 'Cost' : 'C'}, inplace = True)
            
        print(DN)
            
        Pk = pd.merge(Pk, DN, on = ['To'], how = 'left') 
        
        print(f'PK :\n {Pk}')
            
        Pk = Pk.fillna(999) # Pk의 C 값 중 값이 없어 NaN인 값들을 999로 채우기
            #Pk = Pk.sort_values(by = 'To').reset_index(drop = True)
            
        upset = list(Pk[Pk['C1'] > (Pk['C'] + ac)]['To']) ######### 진짜 여기서 죽는줄알았네 ㅆㅂ
        print(upset)
        #print(upset)
        #print(Pk)
            
        if len(upset) > 0:
            Pk.loc[Pk['To'].isin(upset), 'Fin'] = 0
            Pk.loc[Pk['To'].isin(upset), 'C1'] = Pk.loc[Pk['To'].isin(upset), 'C'] + ac
            Pk.loc[Pk['To'].isin(upset), 'From'] = a
            
        Pk = Pk.iloc[:, 0:-1]
        Pk = Pk.sort_values(by = ['Fin', 'C1', 'To']).reset_index(drop = True)
        print('=' * 40)
    
    out = pd.DataFrame({
        'O' : Pk['From'],
        'D' : Pk['To'],
        'SP' : Pk['C1']
    })
    
    out = out.sort_values(by = 'SP').reset_index(drop = True)
    
    return out

In [58]:
Dijkstra(1, net1)

PK :
    To  From   C1  Fin
0   1     0    0    0
1   3     0  999    0
2   4     0  999    0
1
PK :
    To  From   C1  Fin
0   1     0    0    1
1   3     0  999    0
2   4     0  999    0
1 2
   O  D  FFT  alpha  Capacity  beta  Length  Volume  Cost  link_num
0  1  2    1      4         1  0.15       1       0   1.0         1
1  1  3    1      4         3  0.15       1       0   1.0         2
   To    C
0   2  1.0
1   3  1.0
PK :
    To  From   C1  Fin    C
0   1     0    0    1  NaN
1   3     0  999    0  1.0
2   4     0  999    0  NaN
[3]
3
PK :
    To  From     C1  Fin
0   1     0    0.0    1
1   3     1    1.0    1
2   4     0  999.0    0
3 4
   O  D  FFT  alpha  Capacity  beta  Length  Volume  Cost  link_num
2  3  2    1      4         2  0.15       1       0   1.0         3
3  3  4    1      4         4  0.15       1       0   1.0         4
   To    C
2   2  1.0
3   4  1.0
PK :
    To  From     C1  Fin    C
0   1     0    0.0    1  NaN
1   3     1    1.0    1  NaN
2   4     0  

,O,D,SP
0,0,1,0.0
1,1,3,1.0
2,3,4,2.0


# Shortest Path of Dijkstra

In [25]:
def SPD(a, b, net1):
    mintree = Dijkstra(a, net1)
    out = mintree[mintree['D'] == b]
    
    while True:
        if out['O'].iloc[0] == a:
            break
        
        added = mintree[mintree['D'] == out['O'].iloc[0]]
        out = pd.concat([added, out], ignore_index = True)
        #print(out)
    
    return out

In [32]:
Dijkstra(4, net1)

,O,D,SP
0,0,4,0
1,0,1,999
2,0,3,999
